# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 5. Klasifikasi Decision Tree


### 1) Import Data

Praktikum kali ini menggunakan dataset [Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) dari UCI Machine Learning Repository. Dataset ini telah digunakan pada praktikum sebelumnya. Detail fitur dapat Anda pelajari pada link yang tersedia.

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [ ]:
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1RXcXrwkVoOI-hPvfJM1-FlmO_bB7M3YJ" -O car_sample.csv

--2023-03-30 17:15:55--  https://drive.google.com/uc?export=download&id=1RXcXrwkVoOI-hPvfJM1-FlmO_bB7M3YJ
Resolving drive.google.com (drive.google.com)... 142.251.163.100, 142.251.163.102, 142.251.163.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.163.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bdo4ilpodamngiol2fitn1r2ehcfe5mn/1680196500000/13422030100481849639/*/1RXcXrwkVoOI-hPvfJM1-FlmO_bB7M3YJ?e=download&uuid=27930393-0aa0-4863-baf3-9b7d91d9393c [following]
--2023-03-30 17:15:56--  https://doc-0k-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bdo4ilpodamngiol2fitn1r2ehcfe5mn/1680196500000/13422030100481849639/*/1RXcXrwkVoOI-hPvfJM1-FlmO_bB7M3YJ?e=download&uuid=27930393-0aa0-4863-baf3-9b7d91d9393c
Resolving doc-0k-a8-docs.googleusercontent.com (doc-0k-a8-docs.googleusercontent.com)... 142.251.163.132, 

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('car_sample.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [ ]:
data.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**. Agar pengacakan data dilakukan secara konstan, parameter **random_state** diisi dengan nilai integer tertentu, pada praktikum ini diset 101. Kemudian, nilai indeks pada data latih dan data uji diatur ulang agar berurutan nilainya


In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2,random_state=101)
data_latih.reset_index(drop=True)
data_uji.reset_index(drop=True)

,buying,maint,lug_boot,safety,class
0,vhigh,high,med,med,unacc
1,low,med,small,high,unacc
2,vhigh,low,big,med,unacc
3,low,vhigh,small,med,unacc
4,med,low,small,low,unacc
...,...,...,...,...,...
341,high,med,med,med,unacc
342,low,med,big,med,good
343,vhigh,vhigh,big,med,unacc
344,vhigh,low,med,low,unacc


Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 208 data, dan **data_uji** terdiri dari 52 data

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

346
1382


## 3) Menghitung Gini

Nilai Gini merupakan salah satu kriteria penentu variabel apa yang akan digunakan untuk membentuk cabang pada decision tree. Variabel dengan nilai Gini terbesar akan digunakan sebagai pembentukan cabang

Buatlah fungsi bernama **hitung_gini** yang berfungsi menghitung nilai Gini dari suatu nilai pada sebuah variabel

In [ ]:
def hitung_gini(kolom_kelas):
  elemen,banyak = np.unique(kolom_kelas,return_counts = True)
  nilai_gini = 1 - np.sum([(banyak[i]/np.sum(banyak))**2 for i in range (len(elemen))])
  return nilai_gini

Buatlah fungsi bernama **gini_split** yang digunakan untuk menghitung nilai Gini keseluruhan dari sebuah variabel.

In [ ]:
def gini_split(data,nama_fitur_split,nama_fitur_kelas):
  nilai,banyak = np.unique(data[nama_fitur_split], return_counts=True)
  gini_split = np.sum([(banyak[i]/np.sum(banyak))*hitung_gini(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas])for i in range (len(nilai))])
  return gini_split

Ujilah fungsi **gini_split** menggunakan data_latih pada variabel **buying** dan variabel kelas bernama **class**.

In [ ]:
gini_split(data_latih,"buying","class")

0.4498424838345615

## 4) Pembentukan pohon

Pembentukan pohon dilakukan secara rekursif. Seperti metode rekursif pada umumnya, perlu ditentukan kondisi berhenti terlebih dahulu. Kondisi berhenti pada pembentukan pohon adalah:


1.   Jika hanya ada satu kelas pada data, kembalikan kelas tersebut
2.   Jika fitur data  = 0 (tidak ada fitur yang tersisa), kembalikan kelas dari parent
3. Jika data kosong (tidak ada data), kembalikan kelas dengan frekuensi terbanyak

Selain kondisi berhenti tersebut, dilakukan pembentukan pohon secara rekursif menggunakan fungsi **buat_tree**.



In [ ]:
def buat_tree(data,data_awal,daftar_fitur,nama_fitur_kelas,kelas_parent_node=None):
  #satu kelas pada data
  if len(np.unique(data[nama_fitur_kelas])) <=1:
    return np.unique(data[nama_fitur_kelas]) [0]
  #data kosong
  elif len(data)==0:
    return np.unique (data_awal[nama_fitur_kelas]) [np.argmax(np.unique(data_awal[nama_fitur_kelas],return_counts=True)[1])]
  #tidak ada fitur yang tersisa
  elif len(daftar_fitur) == 0:
    return kelas_parent_node
  else:
    kelas_parent_node = np.unique(data[nama_fitur_kelas]) [np.argmax(np.unique(data[nama_fitur_kelas],return_counts=True)[1])]
    nilai_split = [gini_split(data,fitur,nama_fitur_kelas) for fitur in daftar_fitur]
    index_fitur_terbaik = np.argmin(nilai_split)
    fitur_terbaik = daftar_fitur [index_fitur_terbaik]
    tree = {fitur_terbaik:{}}
    daftar_fitur = [i for i in daftar_fitur if i != fitur_terbaik]
    for nilai in np.unique(data[fitur_terbaik]):
        sub_data = data.where(data[fitur_terbaik]==nilai).dropna()
        subtree = buat_tree(sub_data,data_awal,daftar_fitur,nama_fitur_kelas,kelas_parent_node)
        tree[fitur_terbaik][nilai]=subtree
    return(tree)

Buatlah tree menggunakan data latih yang tersedia

In [ ]:
tree = buat_tree(data_latih,data_latih,data_latih.columns[:1], 'class')

Tampilkan tree yang terbentuk. Gunakan library **pprint** untuk menampilkan dictionary secara teratur.

In [ ]:
from pprint import pprint
pprint(tree)

{'buying': {'high': 'unacc', 'low': 'unacc', 'med': 'unacc', 'vhigh': 'unacc'}}


## 5) Proses prediksi

Proses prediksi kelas pada data uji dilakukan dengan melakukan *tree traversal* sampai menemui leaf.

In [ ]:
def prediksi(data_uji,tree):
  for key in list(data_uji.keys()):
    if key in list(tree.keys()):
      try:
        hasil = tree[key][data_uji[key]]
      except:
        return 1
      hasil = tree[key][data_uji[key]]
      if isinstance(hasil,dict):
        return prediksi(data_uji,hasil)
      else:
        return hasil

## 6) Proses Pengujian
Lakukan pengujian menggunakan data uji. Kelas pada data uji perlu dihapus dan data uji perlu diubah menjadi dictionary

In [ ]:
data_uji_dict = data_uji.iloc[:,:-1].to_dict(orient = "records")

Lakukan pengujian terhadap keseluruhan data uji menggunakan looping.

In [ ]:
hasil_prediksi_total = []
for i in range(len(data_uji_dict)):
  hasil_prediksi = prediksi(data_uji_dict[i],tree)
  hasil_prediksi_total.append(hasil_prediksi)

Bandingkan hasil prediksi dengan label sebenarnya. Hitunglah banyaknya data uji yang memiliki kelas prediksi sama dengan kelas sebenarnya

In [ ]:
print("Total prediksi benar: ",sum(hasil_prediksi_total==data_uji['class']))

Total prediksi benar:  252


## TUGAS
Pada tugas kali ini Anda diminta memodifikasi metode pembentukan tree yang telah Anda agar metode tersebut menggunakan information gain sebagai dasar percabangan. Lengkapilah kerangka source code di bawah ini

Lengkapi fungsi hitung_entropy

In [ ]:
def hitung_entropy(kolom_kelas):
  #tuliskan kode Anda di sini
  elements, counts = np.unique(kolom_kelas, return_counts = True)
  entropy = np.sum([(-counts[i]/np.sum(counts)) * np.log2(counts[i] / np.sum(counts)) for i in range(len(elements))])
  return entropy

Lengkapi fungsi information_gain

In [ ]:
def information_gain(data, nama_fitur_split, nama_fitur_kelas):
  #tuliskan kode Anda di sini
  entropy_total = hitung_entropy(data[nama_fitur_kelas])
  values, counts = np.unique(data[nama_fitur_split], return_counts = True)
  entropy_weighted = np.sum([(counts[i] / np.sum(counts)) * hitung_entropy(data.where(data[nama_fitur_split] == values[i]).dropna() [nama_fitur_kelas]) for i in range(len(values))])
  information_gain = entropy_total - entropy_weighted
  return information_gain

Lengkapi fungsi **buat_tree_ig**. Isinya sama persis dengan fungsi **buat_tree**, hanya saja penghitungan **gini_split** diganti dengan **information_gain**. Selain itu, percabangan dilakukan dengan menggunakan nilai **information_gain** **terbesar**

In [ ]:
def buat_tree_ig(data,data_awal, daftar_fitur, nama_fitur_kelas,kelas_parent_node=None):
  #tuliskan kode Anda di sini
  if len(np.unique(data[nama_fitur_kelas])) <= 1:
    return np.unique(data[nama_fitur_kelas])[0]
  elif len(data) == 0:
    return np.unique(data_awal[nama_fitur_kelas])[np.argmax(np.unique(data_awal[nama_fitur_kelas], return_counts= True) [1])]
  elif len(daftar_fitur) ==0:
    return kelas_parent_node
  else:
    kelas_parent_node = np.unique(data[nama_fitur_kelas])[np.argmax(np.unique(data[nama_fitur_kelas], return_counts = True) [1])]
    nilai_split = [information_gain(data, fitur, nama_fitur_kelas) for fitur in daftar_fitur]
    index_fitur_terbaik = np.argmax(nilai_split)
    fitur_terbaik = daftar_fitur[index_fitur_terbaik]

    tree = {fitur_terbaik:{}}

    daftar_fitur = [i for i in daftar_fitur if i != fitur_terbaik]

    for nilai in np.unique(data[fitur_terbaik]):
      sub_data = data.where(data[fitur_terbaik] == nilai).dropna()
      subtree = buat_tree(sub_data, data_awal, daftar_fitur, nama_fitur_kelas, kelas_parent_node)
      tree[fitur_terbaik][nilai] = subtree
  return(tree)

Lakukan pembentukan tree menggunakan fungsi **buat_tree_ig**

In [ ]:
tree_ig = buat_tree_ig(data_latih,data_latih,data_latih.columns[:-1],'class')

Tampilkan tree yang terbentuk

In [ ]:
pprint(tree_ig)

{'safety': {'high': {'buying': {'high': {'maint': {'high': {'lug_boot': {'big': 'acc',
                                                                         'med': 'acc',
                                                                         'small': 'acc'}},
                                                   'low': {'lug_boot': {'big': 'acc',
                                                                        'med': 'acc',
                                                                        'small': 'acc'}},
                                                   'med': {'lug_boot': {'big': 'acc',
                                                                        'med': 'acc',
                                                                        'small': 'acc'}},
                                                   'vhigh': 'unacc'}},
                                'low': {'maint': {'high': {'lug_boot': {'big': 'unacc',
                                                    

Lakukan pengujian menggunakan tree yang terbentuk

In [ ]:
hasil_prediksi_total_ig = []
for i in range(len(data_uji_dict)):
  hasil_prediksi = prediksi(data_uji_dict[i],tree_ig)
  hasil_prediksi_total_ig.append(hasil_prediksi)
print("Total prediksi benar: ",sum(hasil_prediksi_total_ig==data_uji['class']))

Total prediksi benar:  251


# PERTANYAAN

Jawablah pertanyaan di bawah ini



1.   Amati tree yang dihasilkan dengan kriteria percabangan GINI dan Information Gain. Apa perbedaan tree yang dihasilkan dari kedua metode tersebut?
2.   Apakah penggunaan Information Gain dapat meningkatkan akurasi prediksi?



Tulis jawaban Anda di cell ini


1.   Pada GINI, total prediksi benarnya adalah 252, sedangkan untuk information gain total prediksi benarnya adalah 251.
2.   Information Gain lebih berfokus pada peningkatan ketepatan pemodelan yang dapat dicapai dengan memilih fitur tertentu. Sedangkan Gini hanya mengukur seberapa seragam data dibagi dengan fitur tertentu.
